In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [13]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [14]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [15]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1])


In [16]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [17]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [19]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [20]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4654,  0.2400, -0.5459, -0.0802,  0.2083,  0.0331, -0.2463, -0.0619,
         -0.2346, -0.1980, -0.0550, -0.0094,  0.4728,  0.6704, -0.4187,  0.2592,
         -0.2874,  0.3682, -0.1638, -0.1899],
        [-0.4432, -0.1936, -0.7735, -0.3086,  0.0841, -0.2309, -0.1879, -0.2749,
         -0.4814, -0.3837, -0.3982,  0.2265,  0.4407,  0.8617, -0.0827,  0.5964,
         -0.0598,  0.2774, -0.4566,  0.0631],
        [-0.4299, -0.2253, -0.6829, -0.4406,  0.1281, -0.0569, -0.3127, -0.1643,
         -0.4352, -0.1513, -0.0035,  0.0987,  0.6658,  0.4820, -0.2003,  0.3261,
         -0.1025,  0.2672, -0.1774, -0.3551]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2400, 0.0000, 0.0000, 0.2083, 0.0331, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.4728, 0.6704, 0.0000, 0.2592, 0.0000, 0.3682,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0841, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.2265, 0.4407, 0.8617, 0.00

In [21]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [22]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [23]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0145, -0.0163, -0.0339,  ..., -0.0038, -0.0023,  0.0273],
        [ 0.0289,  0.0212,  0.0261,  ..., -0.0130,  0.0117, -0.0226]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0253,  0.0120], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0134, -0.0182,  0.0318,  ...,  0.0429, -0.0047,  0.0093],
        [-0.0163, -0.0242, -0.0279,  ...,  0.0403,  0.0042,  0.0172]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 